In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
from matplotlib.colors import LogNorm
from matplotlib import rcParams
import copy

from areposnap.gadget import gadget_readsnap
from areposnap.gadget_subfind import load_subfind

from auriga_basics import *
%matplotlib inline

rcParams['font.size'] = 16
rcParams['lines.linewidth'] = 1.5
rcParams['axes.linewidth'] = 1.5
rcParams['ytick.major.width'] = 1.5
rcParams['xtick.major.width'] = 1.5
rcParams['image.cmap'] = 'magma'

cmap = copy.copy(plt.cm.get_cmap(rcParams['image.cmap']))
cmap.set_bad((0,0,0))

In [2]:
machine = 'mac'
#machine = 'magny'
machine = 'virgo'

if machine == 'magny':
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
    filedir = "/home/extmilan/masterthesis/files/"
    plotdir = "/home/extmilan/masterthesis/plots/"
elif machine == 'mac': 
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
    filedir = "/Users/smilanov/Documents/masterthesis/auriga_files/files/"
    plotdir = "/Users/smilanov/Documents/masterthesis/auriga_files/plots/"
elif machine == 'virgo': 
    basedir = "/virgo/simulations/Auriga/level4_MHD/"
    filedir = "/u/milas/masterthesis/masterproject/files/"
    plotdir = "/u/milas/masterthesis/masterproject/plots/"


In [3]:
#### path = /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/*
level = 4

j = 0
for halo_number in [24]:  # range(1, 31):
    halodir = basedir+"halo_{0}/".format(halo_number)
    snappath = halodir+"output/"
    for snapnr in range(127,128,1):
        print("level   : {0}".format(level))
        print("halo    : {0}".format(halo_number))
        print("snapnr  : {0}".format(snapnr))
        print("basedir : {0}".format(basedir))
        print("halodir : {0}".format(halodir))
        print("snappath: {0}\n".format(snappath))
        s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[1,2,3,4], 
            haloid=0, galradfac=0.1, verbose=True) 

        # Clean negative and zero values of gmet to avoid RuntimeErrors
        # later on (e.g. dividing by zero)
        s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )
     

level   : 4
halo    : 24
snapnr  : 127
basedir : /virgo/simulations/Auriga/level4_MHD/
halodir : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath: /virgo/simulations/Auriga/level4_MHD/halo_24/output/

Found 1783163 stars.
Rotated pos.
Rotated vel.

galrad  : 0.02408556640148163
redshift: 2.220446049250313e-16
time    : 0.9999999999999998
center  : [ 0.  0.  0.]



In [30]:
i_dm = np.where(s.type == 1)

In [32]:
i_stars = np.where(s.type == 4)

In [33]:
1000* np.max(s.pos[i_stars])

43582.149505615234

In [27]:
print(np.sum(i_stars), np.sum(i_dm))

0 869605


In [4]:
s.cosmology_get_lookback_time_from_a( s.time, is_flat=True )

2.5673723560210484e-15

In [ ]:
### plot all stars in Simulation 24
istars, = np.where( (s.type == 4) )#& (s.halo == 0) )#& (s.subhalo == 0))
(x, y, z), (vx, vy, vz), rxyz, rxy = get_cartesian_vectors(s, sf, istars)

binnum = 401
H1, xedges1, yedges1 = np.histogram2d(x, y, bins=binnum)
H2, xedges2, yedges2 = np.histogram2d(rxy, z, bins=binnum)
vmin = 1e0
if np.max(H1) > np.max(H2):
    vmax = np.max(H1)
else: vmax = np.max(H2)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
h1 = ax1.hist2d(x, y, bins=binnum, norm=matplotlib.colors.LogNorm(), cmap = 'inferno', vmin = vmin, vmax = vmax)
h2 = ax2.hist2d(rxy, z, bins=binnum, norm=matplotlib.colors.LogNorm(), cmap = 'inferno', vmin = vmin, vmax= vmax)
ax1.set_xlabel("x [kpc]")
ax1.set_ylabel("y [kpc]")
ax1.set_aspect('equal')
ax2.set_xlabel("R [kpc]")
ax2.set_ylabel("z [kpc]")
ax2.set_aspect('equal')
cbar2 = fig.colorbar(h2[3], ax = ax2 )


In [ ]:
### plot stars of halo 0 in Simulation 24

istars, = np.where( (s.type == 4) & (s.halo == 0) )#& (s.subhalo == 0))
(x, y, z), (vx, vy, vz), rxyz, rxy = get_cartesian_vectors(s, sf, istars)

binnum = 401
H1, xedges1, yedges1 = np.histogram2d(x, y, bins=binnum)
H2, xedges2, yedges2 = np.histogram2d(rxy, z, bins=binnum)
vmin = 1e0
if np.max(H1) > np.max(H2):
    vmax = np.max(H1)
else: vmax = np.max(H2)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
h1 = ax1.hist2d(x, y, bins=binnum, norm=matplotlib.colors.LogNorm(), cmap = 'inferno', vmin = vmin, vmax = vmax)
h2 = ax2.hist2d(rxy, z, bins=binnum, norm=matplotlib.colors.LogNorm(), cmap = 'inferno', vmin = vmin, vmax= vmax)
ax1.set_xlabel("x [kpc]")
ax1.set_ylabel("y [kpc]")
ax1.set_aspect('equal')
ax2.set_xlabel("R [kpc]")
ax2.set_ylabel("z [kpc]")
ax2.set_aspect('equal')
cbar2 = fig.colorbar(h2[3], ax = ax2 )


In [ ]:
### plot stars of halo 0 within 400kpc in Simulation 24

istars, = np.where( (s.type == 4) & (s.halo == 0) )#& (s.subhalo == 0))
(x, y, z), (vx, vy, vz), rxyz, rxy = get_cartesian_vectors(s, sf, istars)

binnumxx = 401
cmap = copy.copy(plt.cm.inferno)
cmap.set_bad((0,0,0))  # Fill background with black

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
h = ax.hist2d(1000*x, 1000*y, bins=binnumxx, norm=matplotlib.colors.LogNorm(), cmap = cmap, \
           range = [[-400,400], [-400, 400]])
ax.set_xlabel("x [kpc]")
ax.set_ylabel("y [kpc]")
ax.set_aspect('equal')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.2)
fig.colorbar(h[3], cax=cax)
#fig.savefig('../plots/aur_24_xy_distribution.png', dpi = 300, format = 'png', bbox_inches = 'tight')


In [5]:
def read_snap(snapnr, halo_number = 24):
    halodir = basedir+"halo_{0}/".format(halo_number)
    snappath = halodir+"output/"
    
    #print("level   : {0}".format(level))
    #print("halo    : {0}".format(halo_number))
    print("snapnr  : {0}".format(snapnr))
    #print("basedir : {0}".format(basedir))
    #print("halodir : {0}".format(halodir))
    #print("snappath: {0}\n".format(snappath))
    s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
        haloid=0, galradfac=0.1, verbose=True) 

    # Clean negative and zero values of gmet to avoid RuntimeErrors
    # later on (e.g. dividing by zero)
    s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )


    istars, = np.where( (s.type == 4) & (s.halo == 0) )#& (s.subhalo == 0))
    (x, y, z), (vx, vy, vz), rxyz, rxy = get_cartesian_vectors(s, sf, istars)
    snap_time = s.cosmology_get_lookback_time_from_a( s.time, is_flat=True )
    return(x, y, z), (vx, vy, vz), rxyz, rxy, snap_time

In [ ]:
## find min and max values of x, y, z and R in all snapshots to set frame for animation
get_minmaxvals = True
if get_minmaxvals == True:
    startnr = 60
    endnr = 128
    s_snap = np.arange(startnr,endnr,1)
    xmin, xmax, ymin, ymax, zmin, zmax, Rmin, Rmax = [], [], [], [], [], [], [], []
    for j in range(30):
        print(j)
        xmins, xmaxs, ymins, ymaxs, zmins, zmaxs, Rmins, Rmaxs = np.zeros(len(s_snap)), np.zeros(len(s_snap)), np.zeros(len(s_snap)), np.zeros(len(s_snap)), np.zeros(len(s_snap)), np.zeros(len(s_snap)), np.zeros(len(s_snap)), np.zeros(len(s_snap))

        for i in range(len(s_snap)):
            (x, y, z), (vx, vy, vz), rxyz, rxy, snap_time = read_snap(s_snap[i], j+1)
            xmins[i], xmaxs[i] = np.min(x), np.max(x)
            ymins[i], ymaxs[i] = np.min(y), np.max(y)
            zmins[i], zmaxs[i] = np.min(z), np.max(z)
            Rmins[i], Rmaxs[i] = np.min(rxy), np.max(rxy)

        xmin.append(np.min(xmins)) 
        xmax.append(np.min(xmaxs))                                                                                                      
        ymin.append(np.min(ymins)) 
        ymax.append(np.min(ymaxs))                                                                                                        
        zmin.append(np.min(zmins)) 
        zmax.append(np.min(zmaxs))                                                                                                        
        Rmin.append(np.min(Rmins)) 
        Rmax.append(np.min(Rmaxs))                                                                                                         

    print(xmin, xmax, ymin, ymax, zmin, zmax, Rmin, Rmax)
    halonums = np.arange(1,31,1)
    dataout = ([halonums, np.array(xmin), np.array(xmax), np.array(ymin), np.array(ymax), np.array(zmin), np.array(zmax), np.array(Rmin), np.array(Rmax)])  
    np.savetxt(filedir + 'min_max_vals_for_animation_frame_all_simulations.txt', dataout, header='Halo number\txmin [kpc]\txmax [kpc]\tymin [kpc]\tymax [kpc]\tzmin [kpc]\tzmax [kpc]\tRmin [kpc]\tRmax [kpc]')
    

0
snapnr  : 60
Found 243287 stars.
Rotated pos.
Rotated vel.

galrad  : 0.007770749926567078
redshift: 2.1032696525957713
time    : 0.32224076923625916
center  : [ 0.  0.  0.]

snapnr  : 61
Found 265117 stars.
Rotated pos.
Rotated vel.

galrad  : 0.008071627467870712
redshift: 2.0020281392528516
time    : 0.3331081367707903
center  : [ 0.  0.  0.]

snapnr  : 62
Found 286825 stars.
Rotated pos.
Rotated vel.

galrad  : 0.008416595309972763
redshift: 1.9040895435327672
time    : 0.344341999449342
center  : [ 0.  0.  0.]

snapnr  : 63
Found 304113 stars.
Rotated pos.
Rotated vel.

galrad  : 0.00867537260055542
redshift: 1.822689252620354
time    : 0.3542720825792927
center  : [ 0.  0.  0.]

snapnr  : 64
Found 333235 stars.
Rotated pos.
Rotated vel.

galrad  : 0.008959954977035523
redshift: 1.7435705743308647
time    : 0.3644885279628326
center  : [ 0.  0.  0.]

snapnr  : 65
Found 351424 stars.
Rotated pos.
Rotated vel.

galrad  : 0.009256134182214737
redshift: 1.6666695561144653
time    : 

Found 1269105 stars.
Rotated pos.
Rotated vel.

galrad  : 0.019089092314243317
redshift: 0.2733533465784399
time    : 0.7853279709729014
center  : [ 0.  0.  0.]

snapnr  : 108
Found 1280096 stars.
Rotated pos.
Rotated vel.

galrad  : 0.019189572334289553
redshift: 0.2613432561610123
time    : 0.7928056023731169
center  : [ 0.  0.  0.]

snapnr  : 109
Found 1278636 stars.
Rotated pos.
Rotated vel.

galrad  : 0.019359827041625977
redshift: 0.24354018155467028
time    : 0.8041557601699713
center  : [ 0.  0.  0.]

snapnr  : 110
Found 1283048 stars.
Rotated pos.
Rotated vel.

galrad  : 0.019513048231601715
redshift: 0.22598838626019768
time    : 0.8156684118765909
center  : [ 0.  0.  0.]

snapnr  : 111
Found 1285963 stars.
Rotated pos.
Rotated vel.

galrad  : 0.019634324312210086
redshift: 0.21442503551449454
time    : 0.823434934850752
center  : [ 0.  0.  0.]

snapnr  : 112
Found 519915 stars.
Rotated pos.
Rotated vel.

galrad  : 0.017871522903442384
redshift: 0.19728418237600986
time    : 

snapnr  : 86
Found 1121003 stars.
Rotated pos.
Rotated vel.

galrad  : 0.018449503183364867
redshift: 0.7326361820223115
time    : 0.5771552102951079
center  : [ 0.  0.  0.]

snapnr  : 87
Found 1139001 stars.
Rotated pos.
Rotated vel.

galrad  : 0.018706731498241425
redshift: 0.7001063537185233
time    : 0.5881984958251407
center  : [ 0.  0.  0.]

snapnr  : 88
Found 1148623 stars.
Rotated pos.
Rotated vel.

galrad  : 0.018879401683807376
redshift: 0.6761104112134777
time    : 0.596619407235837
center  : [ 0.  0.  0.]

snapnr  : 89
Found 1203804 stars.
Rotated pos.
Rotated vel.

galrad  : 0.019137924909591677
redshift: 0.6446418406845371
time    : 0.6080351206337894
center  : [ 0.  0.  0.]

snapnr  : 90
Found 1245660 stars.
Rotated pos.
Rotated vel.

galrad  : 0.019318880140781404
redshift: 0.6214287452425136
time    : 0.616740021992414
center  : [ 0.  0.  0.]

snapnr  : 91
Found 1238192 stars.
Rotated pos.
Rotated vel.

galrad  : 0.01951500028371811
redshift: 0.5985432881875667
time   

Found 320086 stars.
Rotated pos.
Rotated vel.

galrad  : 0.009242361038923263
redshift: 1.6666695561144653
time    : 0.3749995936718436
center  : [ 0.  0.  0.]

snapnr  : 66
Found 327948 stars.
Rotated pos.
Rotated vel.

galrad  : 0.009525830298662186
redshift: 1.6042345220731056
time    : 0.3839899945738943
center  : [ 0.  0.  0.]

snapnr  : 67
Found 192423 stars.
Rotated pos.
Rotated vel.

galrad  : 0.01001202240586281
redshift: 1.5312390291576135
time    : 0.395063440663206
center  : [ 0.  0.  0.]

snapnr  : 68
Found 358340 stars.
Rotated pos.
Rotated vel.

galrad  : 0.011891089379787445
redshift: 1.4719748452657977
time    : 0.40453486082803386
center  : [ 0.  0.  0.]

snapnr  : 69
Found 372683 stars.
Rotated pos.
Rotated vel.

galrad  : 0.012661144137382507
redshift: 1.4140982203725216
time    : 0.41423335287728646
center  : [ 0.  0.  0.]

snapnr  : 70
Found 561957 stars.
Rotated pos.
Rotated vel.

galrad  : 0.01313217878341675
redshift: 1.3575766674029972
time    : 0.424164360729

In [ ]:
startnr = 60
endnr = 128

norm=matplotlib.colors.LogNorm()
s_snap = np.arange(startnr,endnr,1)


xminlim, xmaxlim, yminlim, ymaxlim = int(xmin)-10, int(xmax)+10, int(ymin)-10, int(ymax)+10 
zminlim, zmaxlim, Rminlim, Rmaxlim = int(zmin)-10, int(zmax)+10, int(Rmin)-10, int(Rmax)+10
### for xy plot ###
(x, y, z), (vx, vy, vz), rxyz, rxy, snap_time = read_snap(s_snap[0])
X = x
Y = y
X = np.append(X, [xminlim, xmaxlim])
Y = np.append(Y, [yminlim, ymaxlim])

fig, ax = plt.subplots(figsize = (8,8))
ax.set_ylim([-400, 400])
ax.set_xlim([-400, 400])
#Create 2d Histogram
data,xedges,yedges = np.histogram2d(X,Y, bins = 501)
#data_bg,x_bg,y_bg = np.histogram2d(X_bg,Y_bg, bins = 501)


#Smooth with filter
ext = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
print(ext)
im = plt.imshow(data.T, origin = 'lower', interpolation = 'gaussian', cmap = cmap, norm=norm, extent = ext, animated=True, )
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]')
ax.set_aspect('equal')
time_text = ax.text(0.1, 0.9,'', color = 'white', transform=ax.transAxes, fontsize=16)

#Define animation. 
def animate(i) :
    (x, y, z), (vx, vy, vz), rxyz, rxy, snap_time = read_snap(s_snap[i])
    X = x
    Y = y
    X = np.append(X, [xminlim, xmaxlim])
    Y = np.append(Y, [yminlim, ymaxlim])    
    #X = np.append(X, [-201, 201])
    #Y = np.append(Y, [-201, 201])
    data,xedge,yedge = np.histogram2d(X,Y, bins = 501)
    
    im.set_data(data)
    
    time_text.set_text("Lookback time: {:.2f} Gyr".format(snap_time) )
    

anim_xy = animation.FuncAnimation(fig, animate, frames=len(s_snap))

plt.show()

anim_xy.save('../plots/xy_evolution.gif',writer='imagemagick',fps=2)




In [14]:
print(xmin, xmax, ymin, ymax, zmin, zmax, Rmin, Rmax)

-849.114990234 459.571899414 -816.423339844 439.729125977 -330.904510498 318.737487793 0.0 863.537353516


In [ ]:
infile_minmax = filedir + 'min_max_vals_for_animation_frame_all_simulations.txt'
halnum, xmin, xmax, ymin, ymax, zmin, zmax, Rmin, Rmax = np.loadtxt(infile_minmax, unpack = True)
print(halnum)